In [ ]:
import pandas as pd 
import numpy as np 
#import seaborn as sns
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import dask.dataframe as dd
# dask.dataframe as dd
#import dask.array as da
#import dask.bag as dg
main_directory = os.getcwd()
target_directory = "ml-latest-small"
work_directory = os.path.join(main_directory, target_directory)

movies = dd.read_csv(os.path.join(work_directory, "movies.csv"))
ratings = dd.read_csv(os.path.join(work_directory,'ratings.csv'))
tags = pd.read_csv(os.path.join(work_directory, 'tags.csv'))
movies.head()
ratings.head()
ratings = ratings.categorize(columns=['userId'])
final_dataset = ratings.pivot_table(index='movieId', columns='userId', values='rating').compute()
#ratings.head()
final_dataset.fillna(0)
#final_dataset.head()
num_user_voted = ratings.groupby('movieId')['rating'].agg('count').compute()  #votes for each movie
num_movies_voted = ratings.groupby('userId')['rating'].agg('count').compute() #votes by each user
#num_movies_voted.sort_values()
num_movies_voted.compute()
#movie_idx = final_dataset.iloc[val[0]]['movieId']
f,ax  = plt.subplots(1,1,figsize=(16,4))
plt.scatter(num_user_voted.index, num_user_voted, color='mediumseagreen')
plt.axhline(y=10, color='r')
plt.xlabel('MovieId')
plt.ylabel('Number of users Voted')
plt.show()

# A movie should have at least 10 user ratings 

# A user should have rated at least 50 movies

final_dataset = final_dataset.loc[num_user_voted[num_user_voted > 10].index, :]
f,ax = plt.subplots(1,1, figsize=(16,4))
plt.scatter(num_movies_voted.index, num_movies_voted, color='mediumseagreen')
plt.axhline(y=50, color='r')
plt.xlabel('UserId')
plt.ylabel('No of votes by user')
plt.show()
final_dataset  = final_dataset.loc[:, num_movies_voted[num_movies_voted>50].index]
final_dataset
# example of how  sparsity works 
sample = np.array([[0,0,3,0,0], [4,0,0,0,2], [0,0,0,0,1]])
sparsity = 1.0 - (np.count_nonzero(sample) / float(sample.size))
print(sparsity)
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)
csr_data
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=20, n_jobs=-1)
model.fit(csr_data)
import pickle

data = {"model": model}
with open('saved_steps.pkl','wb') as file:
    pickle.dump(data,file)
with open('saved_steps.pkl', 'rb') as file:
    data = pickle.load(file)
    
knn = data['model']
def get_movie_recommendation(movie_name):
    movie_name = movie_name.lower()
    n_movies_to_recommend = 5
    movies['title1'] = movies['title'].str.lower()
    movie_list = movies[movies['title1'].str.contains(movie_name)]
    if len(movie_list):
        movie_idx = movie_list.iloc[0]['movieId']
        movie_idx = final_dataset[final_dataset['movieId']== movie_idx].index[0]
        distances, indices = knn.kneighbors(csr_data[movie_idx], n_neighbors=n_movies_to_recommend+1)
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())),key=lambda x:x[1])[:0:-1]
        recommend_frame  = []
        for val in rec_movie_indices:
            movie_idx = final_dataset.iloc[val[0]]['movieId']
            idx = movies[movies['movieId']== movie_idx].index
            
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Genre':movies.iloc[idx]['genres'].values[0]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_recommend+1))
        return df
    
    else:
        return 'No  movies found. Please Check for another movie'
get_movie_recommendation("Iron man")

def get_most_watched(n):
    most_watched = []
    most_watched_movies = num_user_voted.sort_values().tail(n).index.to_list()
    for val in most_watched_movies:
        idx_1 = movies[movies['movieId']== val].index
        name = movies.iloc[idx_1]['title'].values[0]
        most_watched.append({'Title':name})
    df_most_watched = pd.DataFrame(most_watched,index=range(1,n+1))
    return df_most_watched
get_most_watched(10)
count_group = ratings.groupby("movieId").count()["rating"]

movie_list = count_group[count_group > 10].index.values
movie_list[:5]
